In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

# === CONFIGURATION ===
npy_file_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\1.npy"  # Change this to your actual .npy file name if needed
output_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\Visualization"
gif_filename = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\Visualization\velocity_animation.gif"

# === LOAD DATA ===
velocity_frames = np.load(npy_file_path)  # Expected shape: (30, 192, 192)
os.makedirs(output_dir, exist_ok=True)

# === AUTO-DETECT VELOCITY RANGE ===
vmin = np.min(velocity_frames)
vmax = np.max(velocity_frames)
print(f"Detected velocity range: vmin = {vmin:.2f}, vmax = {vmax:.2f}")
#======
# Example: Plot a frame with highlighted top 5 max and min points
# Flatten all frames to find global top/bottom values
flat_values = velocity_frames.flatten()

# Get indices of top 5 max values
top5_max_indices = np.argpartition(flat_values, -5)[-5:]
top5_max_values = flat_values[top5_max_indices]

# Get indices of bottom 5 min values
top5_min_indices = np.argpartition(flat_values, 5)[:5]
top5_min_values = flat_values[top5_min_indices]

# Convert flat indices back to (frame, y, x) coordinates
max_coords = [np.unravel_index(idx, velocity_frames.shape) for idx in top5_max_indices]
min_coords = [np.unravel_index(idx, velocity_frames.shape) for idx in top5_min_indices]

# Print results
print("Top 5 Maximum Values:")
for val, coord in zip(top5_max_values, max_coords):
    print(f"Value = {val:.2f} at Frame = {coord[0]}, Y = {coord[1]}, X = {coord[2]}")

print("\nTop 5 Minimum Values:")
for val, coord in zip(top5_min_values, min_coords):
    print(f"Value = {val:.2f} at Frame = {coord[0]}, Y = {coord[1]}, X = {coord[2]}")


fig, ax = plt.subplots(figsize=(4, 4))
frame_idx = 15  # Choose one frame to display

# Show the selected frame
im = ax.imshow(velocity_frames[frame_idx], cmap='seismic', vmin=np.min(velocity_frames), vmax=np.max(velocity_frames))
ax.set_title(f"Frame {frame_idx}")

# Plot max points that are in this frame
for val, (f, y, x) in zip(top5_max_values, max_coords):
    if f == frame_idx:
        ax.plot(x, y, 'yo', markersize=6, label='Top Max')

# Plot min points that are in this frame
for val, (f, y, x) in zip(top5_min_values, min_coords):
    if f == frame_idx:
        ax.plot(x, y, 'go', markersize=6, label='Top Min')

# Add colorbar and legend
plt.colorbar(im, ax=ax, shrink=0.7)
ax.legend()
plt.show()
# === GENERATE COLOR-MAPPED PNGs ===
frame_paths = []

for i, frame in enumerate(velocity_frames):
    fig, ax = plt.subplots(figsize=(3, 3))

    # Display the velocity frame with a diverging colormap
    im = ax.imshow(frame, cmap='seismic', vmin=vmin, vmax=vmax)
    ax.set_title(f"Frame {i}", fontsize=10)
    ax.axis("off")

    # Add a velocity colorbar
    cbar = plt.colorbar(im, ax=ax, shrink=0.75, pad=0.02)
    cbar.set_label("Velocity", fontsize=8)

    # Save frame to file
    frame_path = os.path.join(output_dir, f"frame_{i:02d}.png")
    plt.savefig(frame_path, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    frame_paths.append(frame_path)

# === CREATE GIF FROM SAVED FRAMES ===
images = [imageio.v2.imread(p) for p in frame_paths]
imageio.mimsave(gif_filename, images, duration=0.3)

print(f"GIF saved as: {gif_filename}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data
# Load velocity data
npy_file_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\4_raw.npy"  # Change this to your actual .npy file name if needed

velocity_frames = np.load(npy_file_path)  # Expected shape: (30, 192, 192)
vmin, vmax = np.min(velocity_frames), np.max(velocity_frames)

# Top 5 max/min
flat = velocity_frames.flatten()
top5_max_idx = np.argpartition(flat, -5)[-5:]
top5_min_idx = np.argpartition(flat, 5)[:5]
top5_max_coords = [np.unravel_index(i, velocity_frames.shape) for i in top5_max_idx]
top5_min_coords = [np.unravel_index(i, velocity_frames.shape) for i in top5_min_idx]
top5_max_values = flat[top5_max_idx]
top5_min_values = flat[top5_min_idx]

grouped_coords = {}
print(" Top Velocity Extremes (Max & Min):\n")
for i, (val, coord) in enumerate(zip(top5_max_values, top5_max_coords)):
    f, y, x = coord
    grouped_coords.setdefault(f, []).append(("Max", i + 1, val, y, x))
    print(f"Max #{i+1}: Value = {val:.2f} at Frame = {f}, Y = {y}, X = {x}")

for i, (val, coord) in enumerate(zip(top5_min_values, top5_min_coords)):
    f, y, x = coord
    grouped_coords.setdefault(f, []).append(("Min", i + 1, val, y, x))
    print(f"Min #{i+1}: Value = {val:.2f} at Frame = {f}, Y = {y}, X = {x}")

# Group by frame
for i, (val, coord) in enumerate(zip(top5_max_values, top5_max_coords)):
    f, y, x = coord
    grouped_coords.setdefault(f, []).append(("Max", i + 1, val, y, x))
for i, (val, coord) in enumerate(zip(top5_min_values, top5_min_coords)):
    f, y, x = coord
    grouped_coords.setdefault(f, []).append(("Min", i + 1, val, y, x))

# Display only affected frames
for f, points in sorted(grouped_coords.items()):
    fig, ax = plt.subplots(figsize=(5, 5))
    
    # Extract frame
    frame = velocity_frames[f]
    
    # Create masked array: only show non-zero values
    masked = np.ma.masked_where(frame == 0, frame)

    # White background
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')

    # Show only non-zero with colormap
    im = ax.imshow(masked, cmap='jet', vmin=vmin, vmax=vmax)

    # Title
    ax.set_title(f"Frame {f} – Jet Colormap over Masked Region", fontsize=11)

    # Annotate points
    for point_type, number, value, y, x in points:
        color = 'white'
        label = f"{point_type} #{number}\n{value:.1f}"
        ax.plot(x, y, 'o', color=color, markersize=6)
        ax.text(x + 2, y, label, color=color, fontsize=8, weight='bold')

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, shrink=0.75, pad=0.02)
    cbar.set_label("Velocity", fontsize=9)

    ax.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import numpy as np

# === INPUT: Folder path ===
input_folder = r"\\isd_netapp\cardiac$\Majid\deepflow\deepFlowDocker\results2"
#P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\inter_subject_BSpline_10
# === Threshold ===
threshold = 150

# === Get list of .npy files ===
all_files = [f for f in os.listdir(input_folder) if f.endswith(".npy")]

# === Process in batches of 10 ===
batch_size = 10
total_files = len(all_files)

for start in range(0, total_files, batch_size):
    end = min(start + batch_size, total_files)
    batch = all_files[start:end]
    matching_files = []

    for filename in batch:
        file_path = os.path.join(input_folder, filename)
        try:
            arr = np.load(file_path)
            max_val = np.max(arr)
            if max_val < threshold:
                matching_files.append((filename, max_val))
        except Exception as e:
            print(f"Could not read {filename}: {e}")

    if matching_files:
        print(f"\nBatch {start + 1} to {end} – Files with max < {threshold}:\n")
        for fname, val in matching_files:
            print(f"{fname} → max = {val:.2f}")
    else:
        continue
        #print(f"\nBatch {start + 1} to {end} – No matching files.\n")



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load velocity data
npy_file_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\3948576_raw.npy"  # Path to the .npy file
velocity_frames = np.load(npy_file_path)  # Expected shape: (30, 192, 192)

# Determine global min and max values for consistent colormap scaling
vmin, vmax = np.min(velocity_frames), np.max(velocity_frames)

# Display all 30 frames
for f in range(velocity_frames.shape[0]):
    fig, ax = plt.subplots(figsize=(5, 5))

    frame = velocity_frames[f]

    # Mask zero values (show only non-zero regions)
    masked = np.ma.masked_where(frame == 0, frame)

    # Set white background
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')

    # Show frame using a colormap
    im = ax.imshow(masked, cmap='jet', vmin=vmin, vmax=vmax)

    # Frame title
    ax.set_title(f"Frame {f}", fontsize=11)

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, shrink=0.75, pad=0.02)
    cbar.set_label("Velocity", fontsize=9)

    ax.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
"""
This script visualizes raw velocity data from a 4D Flow MRI acquisition using a color-coded colormap (jet).
The input is a .npy file containing 30 velocity frames, each of size 192x192, representing a temporal sequence.

For each frame:
- Non-zero velocity regions are identified and cropped with a margin.
- The cropped region is displayed using a jet colormap with a consistent color scale across all frames.
- The center of mass of the non-zero region is used to compute the displacement from the image center (96, 96).
- Each processed frame is saved as a PNG image with a white background and optional annotations.

Finally, all saved images are compiled into an animated GIF for visual inspection.

Note: This script performs pure visualization — no alignment, filtering, or further processing is applied to the input.
"""

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
from PIL import Image  # For GIF creation

# === Load velocity data ===
npy_file_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\vis\3038676_raw.npy"
save_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\3038676_raw_notebook"
os.makedirs(save_dir, exist_ok=True)
gif_path = os.path.join(save_dir, "animated_frames.gif")

# === GIF speed settings ===
frame_duration = 0.2  # Duration per frame in seconds

# === Store filenames for gif creation ===
saved_frame_paths = []

velocity_frames = np.load(npy_file_path)  # Shape: (30, 192, 192)
vmin, vmax = np.min(velocity_frames), np.max(velocity_frames)
image_center = (96, 96)
margin = 10

#=== Generate and save plots ===
for f in range(velocity_frames.shape[0]):
    frame = velocity_frames[f]
    binary_mask = frame != 0

    if not np.any(binary_mask):
        continue

    cy, cx = center_of_mass(binary_mask)
    dy, dx = cy - image_center[0], cx - image_center[1]

    ys, xs = np.where(binary_mask)
    y_min, y_max = max(0, ys.min() - margin), min(192, ys.max() + margin)
    x_min, x_max = max(0, xs.min() - margin), min(192, xs.max() + margin)

    cropped_frame = frame[y_min:y_max, x_min:x_max]
    cropped_masked = np.ma.masked_where(cropped_frame == 0, cropped_frame)

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_facecolor('white')
    fig.patch.set_facecolor('white')

    im = ax.imshow(cropped_masked, cmap='jet', vmin=vmin, vmax=vmax)
    ax.set_title(f"Frame {f} – Δx = {dx:.1f}, Δy = {dy:.1f}", fontsize=10)
    #ax.plot(cx - x_min, cy - y_min, 'wo')
    ax.axis("off")

    cbar = plt.colorbar(im, ax=ax, shrink=0.75, pad=0.02)
    cbar.set_label("Velocity", fontsize=9)

    plt.subplots_adjust(left=0, right=1, top=0.92, bottom=0)

    frame_filename = f"frame_{f:02d}.png"
    save_path = os.path.join(save_dir, frame_filename)
    plt.savefig(save_path, dpi=150)
    plt.close(fig)

    saved_frame_paths.append(save_path)

# === Create GIF with PIL ===
print(f"Creating GIF at: {gif_path}")

images = [Image.open(p).convert("RGB") for p in saved_frame_paths]
images[0].save(
    gif_path,
    save_all=True,
    append_images=images[1:],
    duration=frame_duration * 1000,  # duration in milliseconds
    loop=0
)

print("GIF created successfully.")


In [ ]:
# =============================================================================
# Visualization of Velocity Frames with Circular Mask
# -----------------------------------------------------------------------------
# Use the `tight_crop` flag below to control how each frame is visualized:
#
# - tight_crop = True:
#     Only a tight region around the circular area of interest (radius = 10 pixels
#     centered at (96,96)) is shown. This minimizes surrounding white space and 
#     keeps focus on the circular velocity region. Ideal for clean and compact visual output.
#
# - tight_crop = False:
#     A broader area around the mask is displayed with a fixed margin (e.g. 10 pixels),
#     including parts of the background. Useful when context around the flow region 
#     is important for interpretation.
#
# This flag makes it easy to switch between focused and contextual visualization modes.
# =============================================================================
# This script loads velocity frame data and visualizes each frame by applying a
# fixed circular mask (radius = 10 pixels, centered at image center [96, 96]).
# Values outside the circular mask are displayed in white (masked), ensuring 
# they do not affect visualization or statistics.
#
# Inside the circular mask:
#   - Original velocity values (including zeros) are preserved.
#   - Zero values are visualized as dark blue according to the chosen colormap.
#
# Outside the circular mask:
#   - All velocity values are explicitly set to NaN and displayed as white.
#
# The visualization includes:
#   - Center-of-mass marker (white dot) for each frame.
#   - Cropping around the nonzero masked region plus a margin for clarity.
#   - Saved images are compiled into a GIF animation for easy comparison.
# =============================================================================

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
from PIL import Image  # For GIF creation

# === Load velocity data ===
npy_file_path = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\data\visout_interCircle10\3038676_raw_aligned_velocity_aligned_velocity.npy"
save_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\VisualizationBsCircle10"
os.makedirs(save_dir, exist_ok=True)
gif_path = os.path.join(save_dir, "animated_frames.gif")
# === Visualization settings ===
tight_crop = True  # Set to True for tight circular crop, False to include margin and full background


# === GIF speed settings ===
frame_duration = 0.2  # Duration per frame in seconds

# === Store filenames for gif creation ===
saved_frame_paths = []

velocity_frames = np.load(npy_file_path)  # Shape: (30, 192, 192)
vmin, vmax = np.min(velocity_frames), np.max(velocity_frames)
image_center = (96, 96)
margin = 10
# === Create circular mask ===
yy, xx = np.ogrid[:192, :192]
distance_from_center = np.sqrt((xx - image_center[1])**2 + (yy - image_center[0])**2)
circular_mask = distance_from_center <= 10  # Radius 10 pixels

for f in range(velocity_frames.shape[0]):
    frame = velocity_frames[f]

    # Apply circular mask
    masked_frame = np.where(circular_mask, frame, 0)
    
    # Mask outside circle completely
    display_mask = np.where(circular_mask, masked_frame, np.nan)

    binary_mask = ~np.isnan(display_mask)
    if not np.any(binary_mask):
        continue

    cy, cx = center_of_mass(binary_mask)
    dy, dx = cy - image_center[0], cx - image_center[1]

    ys, xs = np.where(binary_mask)
    y_min, y_max = max(0, ys.min() - margin), min(192, ys.max() + margin)
    x_min, x_max = max(0, xs.min() - margin), min(192, xs.max() + margin)
    if tight_crop:
        # === [TIGHT CROP MODE] Only show circular region around (96,96), radius = 10
        r = 10
        y_min, y_max = image_center[0] - r, image_center[0] + r + 1
        x_min, x_max = image_center[1] - r, image_center[1] + r + 1
        cropped_frame = display_mask[y_min:y_max, x_min:x_max]
        cropped_masked = np.ma.masked_where(np.isnan(cropped_frame), cropped_frame)

        fig, ax = plt.subplots(figsize=(2.5, 2.5))
        ax.set_facecolor('white')
        fig.patch.set_facecolor('white')

        cmap = plt.cm.jet.copy()
        cmap.set_bad(color='white')

        im = ax.imshow(cropped_masked, cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set_title(f"Frame {f}", fontsize=10)
        ax.axis("off")

        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        save_path = os.path.join(save_dir, f"frame_{f:02d}.png")
        plt.savefig(save_path, dpi=150, bbox_inches='tight', pad_inches=0.0)

    else:
        # === [FULL FRAME MODE] Show full region around mask with margin
        ys, xs = np.where(~np.isnan(display_mask))
        y_min = max(0, ys.min() - margin)
        y_max = min(192, ys.max() + margin)
        x_min = max(0, xs.min() - margin)
        x_max = min(192, xs.max() + margin)

        cropped_frame = display_mask[y_min:y_max, x_min:x_max]
        cropped_masked = np.ma.masked_where(np.isnan(cropped_frame), cropped_frame)

        fig, ax = plt.subplots(figsize=(5, 5))
        ax.set_facecolor('white')
        fig.patch.set_facecolor('white')

        cmap = plt.cm.jet.copy()
        cmap.set_bad(color='white')

        im = ax.imshow(cropped_masked, cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set_title(f"Frame {f} – Δx = {dx:.1f}, Δy = {dy:.1f}", fontsize=10)
        #ax.plot(cx - x_min, cy - y_min, 'wo')
        ax.axis("off")
        cbar = plt.colorbar(im, ax=ax, shrink=0.75, pad=0.02)
        cbar.set_label("Velocity", fontsize=9)
        plt.subplots_adjust(left=0, right=1, top=0.92, bottom=0)
        save_path = os.path.join(save_dir, f"frame_{f:02d}.png")
        plt.savefig(save_path, dpi=150)
    
    plt.close(fig)
    saved_frame_paths.append(save_path)

# === Create GIF with PIL ===
print(f"Creating GIF at: {gif_path}")

images = [Image.open(p).convert("RGB") for p in saved_frame_paths]
images[0].save(
    gif_path,
    save_all=True,
    append_images=images[1:],
    duration=frame_duration * 1000,  # duration in milliseconds
    loop=0
)

print("GIF created successfully.")


In [ ]:
# =============================================================================
# Visualization of Mean Velocity Frames with Circular Mask (4x4 Central Window)
# -----------------------------------------------------------------------------
# This script reads multiple velocity frame samples from a directory, computes
# pixel-wise mean frames across all samples, and visualizes each mean frame
# with a circular mask (radius = 10 pixels, centered at [96, 96]).
# Values outside the circular mask are displayed in white.
# Additionally, plots show the average and maximum velocities calculated:
# 1. inside a central 4x4 window
# 2. inside the full circular mask (r=10)
# =============================================================================
# =============================================================================
# Crop Mode Selection for Visualization
# -----------------------------------------------------------------------------
# Set `tight_crop = True` to crop tightly around the circular region of interest.
# Set `tight_crop = False` to include a larger region with margin and background.
# =============================================================================
tight_crop = True  # ⇦ CHANGE THIS TO SWITCH MODES


import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
from PIL import Image
from glob import glob

# === Paths ===
input_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\inter_subject_BSpline_10"
save_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\output\MeanVisualization3"
os.makedirs(save_dir, exist_ok=True)
gif_path = os.path.join(save_dir, "mean_frames.gif")

# === Settings ===
frame_duration = 0.4
num_samples = 10
image_center = (96, 96)
margin = 100
window_size = 4

# === Generate circular mask ===
yy, xx = np.ogrid[:192, :192]
distance = np.sqrt((xx - image_center[1])**2 + (yy - image_center[0])**2)
circular_mask = distance <= 10

# === Central window indices ===
half_window = window_size // 2
window_slice = (slice(image_center[0]-half_window, image_center[0]+half_window),
                slice(image_center[1]-half_window, image_center[1]+half_window))

# === Load data ===
npy_files = glob(os.path.join(input_dir, '*.npy'))[:num_samples]
all_samples = np.array([np.load(f) for f in npy_files])  # (N, 30, 192, 192)
mean_frames = np.mean(all_samples, axis=0)  # (30, 192, 192)

# === Init plots ===
saved_frame_paths = []
avg_velocities = []
max_velocities = []
mean_full_mask = []
max_full_mask = []

vmin, vmax = np.min(mean_frames), np.max(mean_frames)

for f in range(mean_frames.shape[0]):
    frame = mean_frames[f]
    masked_frame = np.where(circular_mask, frame, np.nan)
    binary_mask = ~np.isnan(masked_frame)

    # --- Metrics inside central window ---
    window_values = masked_frame[window_slice]
    avg_velocity = np.nanmean(window_values)
    max_velocity = np.nanmax(window_values)
    avg_velocities.append(avg_velocity)
    max_velocities.append(max_velocity)

    # --- Metrics inside full circular mask ---
    full_mean = np.nanmean(masked_frame)
    full_max = np.nanmax(masked_frame)
    mean_full_mask.append(full_mean)
    max_full_mask.append(full_max)

    # --- Cropping and visualization ---
    cy, cx = center_of_mass(binary_mask)
    dy, dx = cy - image_center[0], cx - image_center[1]
    if tight_crop:
        # Tight crop: just enough to cover the circular region
        y_min, y_max = image_center[0] - 10, image_center[0] + 10 + 1
        x_min, x_max = image_center[1] - 10, image_center[1] + 10 + 1
    else:
        # Loose crop: based on nonzero mask + margin
        ys, xs = np.where(binary_mask)
        y_min, y_max = max(0, ys.min() - margin), min(192, ys.max() + margin)
        x_min, x_max = max(0, xs.min() - margin), min(192, xs.max() + margin)

    cropped = masked_frame[y_min:y_max, x_min:x_max]
    cropped_masked = np.ma.masked_where(np.isnan(cropped), cropped)

    fig, ax = plt.subplots(figsize=(5, 5))
    cmap = plt.cm.jet.copy()
    cmap.set_bad(color='white')
    im = ax.imshow(cropped_masked, cmap=cmap, vmin=vmin, vmax=vmax)


    # === Title and optional colorbar ===
    ax.set_title(f"Mean Frame {f}", fontsize=10)
    #ax.plot(cx - x_min, cy - y_min, 'wo')
    ax.axis("off")
    
    if not tight_crop:
        cbar = plt.colorbar(im, ax=ax, shrink=0.75, pad=0.02)
        cbar.set_label("Mean Velocity", fontsize=9)
        ax.set_title(f"Mean Frame {f} – Δx = {dx:.1f}, Δy = {dy:.1f}", fontsize=10)


    plt.subplots_adjust(left=0, right=1, top=0.92, bottom=0)


    frame_filename = f"mean_frame_{f:02d}.png"
    save_path = os.path.join(save_dir, frame_filename)
    if tight_crop:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', pad_inches=0.0)
    else:
        plt.savefig(save_path, dpi=150)
        plt.close(fig)
    saved_frame_paths.append(save_path)

# === Create GIF ===
images = [Image.open(p).convert("RGB") for p in saved_frame_paths]
images[0].save(
    gif_path,
    save_all=True,
    append_images=images[1:],
    duration=frame_duration * 1000,
    loop=0
)

# === Plot 1: Average & Max inside 4x4 window ===
plt.figure(figsize=(8, 4))
plt.plot(avg_velocities, marker='o', linestyle='-', color='b', label='Mean Velocity (4x4 Window)')
plt.plot(max_velocities, marker='x', linestyle='--', color='r', label='Max Velocity (4x4 Window)')
plt.xlabel('Frame Number')
plt.ylabel('Velocity (cm/s)')
plt.title('Average and Maximum Velocity (4x4 Central Window)')
plt.grid(True)
plt.legend()
# === Two-phase annotation for cardiac cycle ===
ax = plt.gca()

# Systolic Conduit Phase (frames 0-10)
ax.axvspan(0, 10, facecolor='lightblue', alpha=0.3, label='Systolic Conduit Phase')

# Diastolic Reservoir Velocity (frames 10-29)
ax.axvspan(10, 29, facecolor='lightgreen', alpha=0.3, label='Diastolic Reservoir Velocity')

# Add vertical separation line and clear text labels
ax.axvline(x=10, color='black', linestyle='--', linewidth=1)
ax.text(5, plt.ylim()[1]*0.95, 'Systolic Conduit Phase', fontsize=12, fontweight='bold', ha='center', color='blue')
ax.text(19.5, plt.ylim()[1]*0.95, 'Diastolic Reservoir Velocity', fontsize=12, fontweight='bold', ha='center', color='green')

# Adjust legend and layout
ax.legend(loc='upper right', bbox_to_anchor=(1, 0.85))
plt.tight_layout()



plt.tight_layout()
plt.savefig(os.path.join(save_dir, "mean_max_velocity_central_window_plot.png"), dpi=150)
plt.close()

# === Plot 2: Average & Max inside full circular mask ===
plt.figure(figsize=(8, 4))
plt.plot(mean_full_mask, marker='s', linestyle='-', color='g', label='Mean Velocity (Full Mask)')
plt.plot(max_full_mask, marker='d', linestyle='--', color='m', label='Max Velocity (Full Mask)')
plt.xlabel('Frame Number')
plt.ylabel('Velocity (cm/s)')
plt.title('Average and Maximum Velocity (Full Circular Mask r=10)')
plt.grid(True)
plt.legend()

# === Two-phase annotation for cardiac cycle ===
ax = plt.gca()

# Systolic Conduit Phase (frames 0-10)
ax.axvspan(0, 10, facecolor='lightblue', alpha=0.3, label='Systolic Conduit Phase')

# Diastolic Reservoir Velocity (frames 10-29)
ax.axvspan(10, 29, facecolor='lightgreen', alpha=0.3, label='Diastolic Reservoir Velocity')

# Add vertical separation line and clear text labels
ax.axvline(x=10, color='black', linestyle='--', linewidth=1)
ax.text(5, plt.ylim()[1]*0.95, 'Systolic Conduit Phase', fontsize=12, fontweight='bold', ha='center', color='blue')
ax.text(19.5, plt.ylim()[1]*0.95, 'Diastolic Reservoir Velocity', fontsize=12, fontweight='bold', ha='center', color='green')

# Adjust legend and layout
ax.legend(loc='upper right', bbox_to_anchor=(1, 0.85))
plt.tight_layout()


plt.tight_layout()
plt.savefig(os.path.join(save_dir, "mean_max_velocity_full_circle_plot.png"), dpi=150)
plt.close()

print("GIF and velocity plots created successfully.")
